In [7]:
import pandas as pd
import numpy as np
import subprocess
from os import walk
import librosa
import librosa.display
from pydub import AudioSegment

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import initializers
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import load_model

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

In [2]:
# signal2, _ = librosa.load("C:/Users/sdiva/Documents/Python_scripts/RI/audio/yaman_wav/flute.wav")

In [8]:
ROW_DURATION = 10
SAMPLING_RATE=22050
FRAME_SIZE = 2048
HOP = 2048
SAMPLE_ROW_LEN = ROW_DURATION*SAMPLING_RATE
JUMP = SAMPLING_RATE*5
BINS = librosa.fft_frequencies(sr=SAMPLING_RATE, n_fft=FRAME_SIZE)

In [5]:
def create_x_row(signal,sample_row_len,rate,jump,bins):
    """
    Takes audio signal and generates sample rows wrt ROW_DURATION and JUMP
    i.e, if audio signal is of 20 sec, ROW_DURATION = 10 sec, JUMP = 2 sec,
    Total 6 rows will be generated from the audio signal - 0-10 sec, 2-12 sec, ... 10-20 sec 
    i.e of each 10 sec
    """
    num = len(signal)//sample_row_len
    rows = []
    for i in range(0,(num-1)*sample_row_len,jump):
        r = signal[i:i+sample_row_len]
        r_spec = convert_to_spec(r,FRAME_SIZE,HOP)
        max_f = r_spec.argmax(axis=0)
        count_r = np.zeros(len(bins))
        for j in max_f:
            count_r[j]+=1
        rows.append(count_r)
    return rows

def convert_to_spec(row,frame_size,hop):
    stft = librosa.stft(row,n_fft=frame_size, hop_length=hop,center=False)
    spec = np.abs(stft)
    return spec

In [6]:
#Creating dataset from folder paths

paths = ['C:/Users/sdiva/Documents/Python_scripts/RI/audio/train/yaman_wav2/',
         'C:/Users/sdiva/Documents/Python_scripts/RI/audio/train/bhairavi_wav2/']

x_data = []
len_perclass = []
for path in paths:
    f = []
    for (dirpath, dirnames, filenames) in walk(path):
        f.extend(filenames)
        break
    for file in f:
        s, _ = librosa.load(path+file,sr = SAMPLING_RATE)
        row = create_x_row(s,SAMPLE_ROW_LEN,SAMPLING_RATE,JUMP,BINS)
        x_data.extend(row)
    len_perclass.append(len(x_data))


In [9]:
x_data = np.array(x_data)
x_data.shape

(6528, 1025)

In [10]:
#Checking class distribution

len_perclass

[3578, 6528]

In [11]:
#Creating labels : 0 for Yaman, 1 for Bhairavi

y0 = np.zeros(len_perclass[0]).reshape(-1,1)
y1 = np.ones(len_perclass[1]-len_perclass[0]).reshape(-1,1)
y_data = np.vstack((y0,y1))

In [12]:
x_data[0].shape

(1025,)

In [11]:
# import pickle

# with open('C:/Users/sdiva/Documents/Python_scripts/RI/x_data.pickle', 'wb') as x:
#    pickle.dump(x_data, x)

# with open('C:/Users/sdiva/Documents/Python_scripts/RI/y_data.pickle', 'wb') as y:
#    pickle.dump(y_data, y)

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25, stratify=y_data)

In [67]:
input_seq = layers.Input(shape=(1025),dtype='float32')
x = layers.Dense(2, kernel_initializer=initializers.RandomUniform(),activation='relu')(input_seq)
x = layers.Dense(32, kernel_initializer=initializers.RandomUniform(),activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(32, kernel_initializer=initializers.RandomUniform(),activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(32, kernel_initializer=initializers.RandomUniform(),activation='relu')(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(input_seq, preds)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1025)]            0         
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 2052      
_________________________________________________________________
dense_16 (Dense)             (None, 32)                96        
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 32)                1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 32)                1056

In [70]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=200, validation_data=(x_test, y_test))

Epoch 1/200
153/153 [==============================] - 1s 2ms/step - loss: 0.2095 - accuracy: 0.9187 - val_loss: 0.3272 - val_accuracy: 0.8866
Epoch 2/200
153/153 [==============================] - 0s 1ms/step - loss: 0.2072 - accuracy: 0.9203 - val_loss: 0.3264 - val_accuracy: 0.8860
Epoch 3/200
153/153 [==============================] - 0s 1ms/step - loss: 0.2077 - accuracy: 0.9187 - val_loss: 0.3258 - val_accuracy: 0.8866
Epoch 4/200
153/153 [==============================] - 0s 1ms/step - loss: 0.2027 - accuracy: 0.9208 - val_loss: 0.3255 - val_accuracy: 0.8866
Epoch 5/200
153/153 [==============================] - 0s 1ms/step - loss: 0.2019 - accuracy: 0.9201 - val_loss: 0.3251 - val_accuracy: 0.8873
Epoch 6/200
153/153 [==============================] - 0s 1ms/step - loss: 0.2013 - accuracy: 0.9201 - val_loss: 0.3246 - val_accuracy: 0.8873
Epoch 7/200
153/153 [==============================] - 0s 1ms/step - loss: 0.2031 - accuracy: 0.9201 - val_loss: 0.3242 - val_accuracy: 0.8866

In [96]:
#Creating ML classifiers and fitting them

lr = LogisticRegression()
svm = SVC()
xgb = XGBClassifier()

lr.fit(x_train,y_train)
svm.fit(x_train,y_train)
xgb.fit(x_train,y_train)

[20:38:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [15]:
lr_test_pred = lr.predict(x_test)
svm_test_pred = svm.predict(x_test)
xgb_test_pred = xgb.predict(x_test)

lr_acc = accuracy_score(y_test,lr_test_pred)
svm_acc = accuracy_score(y_test,svm_test_pred)
xgb_acc = accuracy_score(y_test,xgb_test_pred)

In [16]:
#Accuracy on test data for ML classifiers

print("LR accuracy: ",lr_acc)
print("SVM accuracy: ",svm_acc)
print("XGB accuracy: ",xgb_acc)

LR accuracy:  0.8952205882352942
SVM accuracy:  0.9656862745098039
XGB accuracy:  0.9675245098039216


In [76]:
#Creating data for testing from completely unkown audio sources

paths_test = ['C:/Users/sdiva/Documents/Python_scripts/RI/audio/test/yaman_test/',
         'C:/Users/sdiva/Documents/Python_scripts/RI/audio/test/bhairavi_test/']

x_data_test = []
len_perclass_test = []
for path in paths_test:
    g = []
    for (dirpath, dirnames, filenames) in walk(path):
        g.extend(filenames)
        break
    for file in g:
        s, _ = librosa.load(path+file,sr = SAMPLING_RATE)
        row = create_x_row(s,SAMPLE_ROW_LEN,SAMPLING_RATE,JUMP,BINS)
        x_data_test.extend(row)
    len_perclass_test.append(len(x_data_test))

x_data_test = np.array(x_data_test)
y0_t = np.zeros(len_perclass_test[0]).reshape(-1,1)
y1_T = np.ones(len_perclass_test[1]-len_perclass_test[0]).reshape(-1,1)
y_data_test = np.vstack((y0_t,y1_T))

In [77]:
#Checking calss distribution

len_perclass_test

[152, 300]

In [83]:
# Predicting based on DNN

pred_data_test = best_model.predict(x_data_test)
pred_data_test = [0 if x < 0.5 else 1 for x in pred_data_test]
y_data_test.shape

(300, 1)

In [84]:
accuracy_score(y_data_test,pred_data_test)

0.6066666666666667

In [ ]:
#Predicitng on unkown test data with ML classifiers

In [80]:
accuracy_score(y_data_test,lr.predict(x_data_test))

0.6

In [81]:
accuracy_score(y_data_test,svm.predict(x_data_test))

0.6466666666666666

In [82]:
accuracy_score(y_data_test,xgb.predict(x_data_test))

0.5966666666666667

In [92]:
def predict_class(signal,sample_row_len,rate,model,jump,bins):
    """
    Given an audio file, function predicts class for the entire file based on the median result of its samples
    i.e, if audio file is of 60 sec, with row-size 10 sec and jump 5 sec, there will be 11 samples of 10 sec each,
    this function will predict class based on the median result of all 11 samples
    """
    num = len(signal)//sample_row_len
    row = []
    for i in range(0,len(signal)-sample_row_len,jump):
        samples = np.array(signal[i:i+sample_row_len])
        r_spec = convert_to_spec(samples,FRAME_SIZE,HOP)
        max_f = r_spec.argmax(axis=0)
        count_r = np.zeros(len(bins))
        for j in max_f:
            count_r[j]+=1
        pred = best_model.predict(count_r.reshape(1,-1))
        row.append(pred)
    med = np.median(row)
    pred_class = 0 if med < 0.5 else 1
    return pred_class


In [93]:
#Loading data for predictions

paths_test = ['C:/Users/sdiva/Documents/Python_scripts/RI/audio/test/yaman_test/',
         'C:/Users/sdiva/Documents/Python_scripts/RI/audio/test/bhairavi_test/']
predictions = []
for path in paths_test:
    g = []
    for (dirpath, dirnames, filenames) in walk(path):
        g.extend(filenames)
        break
    for file in g:
        s, _ = librosa.load(path+file,sr = SAMPLING_RATE)
        pred = predict_class(s,SAMPLE_ROW_LEN,SAMPLING_RATE,model,JUMP,BINS)
        predictions.append(pred)


In [94]:
# paths_test = ['C:/Users/sdiva/Documents/Python_scripts/RI/audio/test/yaman_test/',
#          'C:/Users/sdiva/Documents/Python_scripts/RI/audio/test/bhairavi_test/']

# Total no. audio files in the two test paths is 16, so 16 predictions below. 
# 1-8 are yaman, 9-16 are bhairavi files. 
# 0 -> predicted as yaman, 1 -> predicted as bhairavi

predictions

[0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1]

In [63]:
best_model = load_model("C:/Users/sdiva/Documents/Python_scripts/RI/models/richest_data1.h5")

In [48]:
#Saved the model which get the best predictions for unkown audio files

model.save("C:/Users/sdiva/Documents/Python_scripts/RI/models/richest_data1.h5")

Sources - 

https://www.linkedin.com/pulse/identifying-ragas-carnatic-music-machine-learning-sridhar-ravikoti/

https://www.youtube.com/watch?v=iCwMQJnKk2c&list=PL-wATfeyAMNqIee7cH3q1bh4QJFAaeNv0


